In [ ]:
# Exercise 2: Analyzing network properties

In [ ]:
# Load relevant libraries
# Extensive documentation for iGraph here: http://igraph.org/r/doc/
suppressMessages(library(igraph))

In [ ]:
# Create a network to analyze
# We will discuss the barabasi.game command in more detail later
test.graph <-barabasi.game(100,power=0.5,m=2)
plot.igraph(test.graph)

In [ ]:
# Plot using a layout algorithm and setting parameters to make the 
# network easier to look at
par(mar=c(.1,.1,.1,.1)) # Reduce margins around plot
plot.igraph(test.graph, layout=layout.fruchterman.reingold,
vertex.size=10,         # sets size of the vertex, default is 15
vertex.label.cex=.5,    # size of the vertex label
edge.arrow.size=.5        # sets size of the arrow at the end of the edge
)

In [ ]:
# Overview of the graph
# How large is the network? (This was set when we made the network, but what if it hadn't been?)
 
test.graph      # Returns that it is an IGRAPH object with 100 nodes and 197 links,
                # made with the Barabasi algorithm with parameter settings
                # as well as the edge list up to a certain count
V(test.graph)   # gives the vertex sequence
E(test.graph)   # gives the edge sequence (edge list)

In [ ]:
# Graph properties

# The "GenInd()" function from the NetIndices package takes an adjacency matrix as input
# and returns 10 global properties of the graph
library(NetIndices)

test.graph.adj<-get.adjacency(test.graph,sparse=F)
# in older versions of igraph the default was sparse=F,
# but now you must specify, other wise you get a matrix of 1s and .s
 
test.graph.properties<-GenInd(test.graph.adj)
 
# The function output consists of 10 network properties.
# Five of them are evaluated here:
 
test.graph.properties$N            #number of nodes
 
test.graph.properties$Ltot        #number of links
 
test.graph.properties$LD        #link density (average # of links per node)
 
test.graph.properties$C            #the connectance of the graph
# Connectance is a measure of completeness of a graph, comparing the
# number of observed edges to the number of possible edges
# This function measures connectance as L/(N*(N-1)) where L is links (edges), and N is nodes
# Connectance can also be calculated as L/(N^2)

str(test.graph.properties) # See how GenInd outputs are encoded
names(test.graph.properties) # List all properties provided by GenInd

# More Information avaiable at https://www.rdocumentation.org/packages/NetIndices/versions/1.4.4/topics/GenInd
#N      number of compartments, excluding the externals. 
#T..    total System Throughput. 
#TST    total System Throughflow. 
#Lint   number of Internal links. 
#Ltot   total number of links. 
#LD     link Density. 
#C      connectance (internal). 
#Tijbar average Link Weight. 
#TSTbar average Compartment Throughflow . 
#Cbar   compartmentalization, [0,1], the degree of connectedness of subsystems within a network. 

# References:
# Latham LG. 2006. Network flow analysis algorithms. Ecological Modelling 192: 586-600. 
# Hirata H, Ulanowicz RE. 1984. Informational theoretical analysis of ecological networks. International journal of systems science 15 (3): 261-270 
# Pimm SL, Lawton JH. 1980. Are food webs divided into compartments? Journal of Animal Ecology 49: 879-898. 
# Kones, J.K., Soetaert, K., van Oevelen, D. and J.Owino (2009). Are network indices robust indicators of food web functioning? a Monte Carlo approach. Ecological Modelling, 220, 370-382. 

In [ ]:
# Node degree distributions

# The degree of a node refers to the number of edges associated with a node.
# Degree can be measured as the edges going in ("in degree"), out ("out degree"), or both.
# The degree() function takes a graph input and gives the degree of specified nodes.
# With the argument "v=V(graph)" you tell the function to give the degree of all nodes in the graph,
# while the "mode" argument specifies in, out, or both.
 
in.deg.testgraph<-degree(test.graph,v=V(test.graph),mode="in")
out.deg.testgraph<-degree(test.graph,v=V(test.graph),mode="out")
all.deg.testgraph<-degree(test.graph,v=V(test.graph),mode="all")

# Count & plot the number of times nodes with degree n appear in the graph
all.deg.testgraph
table(all.deg.testgraph)
deg.counts <- table(all.deg.testgraph)
max(deg.counts)
hist(all.deg.testgraph,breaks=50) #,prob=T)
#lines(density(deg.counts)) # Use with prob=T

# Degree distribution is the cumulative frequency of nodes with a given degree
# this, like degree() can be specified as "in", "out", or "all"
deg.distr<-degree.distribution(test.graph,cumulative=T,mode="all")

In [ ]:
# Is the network scale-free?
# We can test this by comparing the node degree distribution to the 
# power-law distribution expected based on the graphs parameters

# Using the power.law.fit() function you can fit a power law to the degree distribution
power<-power.law.fit(all.deg.testgraph)
 
# The output of the power.law.fit() function gives the exponent of the power law ($alpha)
# and the log-likelihood of the parameters used to fit the power law distribution ($logLik)
# Also, it performs a Kolmogov-Smirnov test to determine whether the given degree distribution 
# could have been drawn from the fitted power law distribution.
# The function returns a test statistic ($KS.stat) and p-value ($KS.p) for that test

# Then we can plot the degree distribution
plot(deg.distr,log="xy",
ylim=c(.01,10),
bg="black",pch=21,
xlab="Degree",
ylab="Cumulative Frequency")
 
# And add a line corresponding to the expected power law distribution
lines(1:20,10*(1:20)^((-power$alpha)+1))
 
# Note: The KS test is used to determine if 2 distributions are 
# statistically different, thus the null hypothesis is that the 
# distributions are the same, and the p-value is used to reject
# the null hypothesis of same-ness

# Determine the 'goodness-of-fit' of power law to degree 
# distribution using the Kolmogorov-Smirnov (KS) test
power$KS.p
# p is >> 0.05, therefore these data are likely to follow a power law distribution
# Small p-values (less than 0.05) indicate that the test rejected the hypothesis that the original data 
# could have been drawn from the fitted power-law distribution.

# Note:
# Random graphs typically have a Poisson distribution,
# Preferential attachement random graphs follow a power law
# Many real networks follow a truncated power law degree distribution

In [ ]:
# Graph Diameter
# Diameter is the longest shortest path between two vertices
diameter(test.graph) # Gives the length of the diameter
nodes.diameter<-get.diameter(test.graph) # Gives the labels for each node that participate in the diameter
 
# To visualize the diameter
# First define the node and edge attributes
V(test.graph)$color<-"skyblue" # node default color
V(test.graph)$size<-7 # node default size
V(test.graph)[nodes.diameter]$color<-"darkgreen" # diameter node color
V(test.graph)[nodes.diameter]$size<-10 # diameter node size
V(test.graph)[nodes.diameter]$label.color<-"white" # diameter node label color

E(test.graph)$color<-"grey" # all non-diameter edges will be grey
E(test.graph,path=nodes.diameter)$color<-"darkgreen" # diameter edges will be dark green
E(test.graph)$width<-1
E(test.graph,path=nodes.diameter)$width<-2 # diameter edges will be wider
 
# If you do not set the attributes of all of the nodes and edges then it will
# default such that you only see what you have defined

par(mar=c(.1,.1,.1,.1))
plot.igraph(test.graph,
layout=layout.fruchterman.reingold,
vertex.label.cex=.5,
edge.arrow.size=.5)

In [ ]:
# Properties informative about "small-worldness" of networks

# Clustering coefficient, betweenness, and closeness
# all describe the small world properties of the network.
# A network with small world properties is one in which
# it takes a relatively short path to get from one node to the next
# (e.g., six degrees of separation)

# Clustering coefficient is the proportion of
# a nodes neighbors that can be reached by other neighbors
# in igraph this property is apparently called "transitivity"
 
transitivity(test.graph)
# gives the clustering coefficient of the whole network
 
head(transitivity(test.graph,type="local"))
# gives the clustering coefficient of each node
 
# Betweenness is the number of shortest paths between two nodes that go through each node of interest
graph.betweenness<-betweenness(test.graph,v=V(test.graph))
head(graph.betweenness)

# The edge betweenness score of an edge measures the number of shortest paths through it
graph.edge.betweenness<-edge.betweenness(test.graph,e=E(test.graph))
head(graph.edge.betweenness)
 
# Closeness refers to how connected a node is to its neighbors
graph.closeness<-closeness(test.graph,vids=V(test.graph))
head(graph.closeness)



In [ ]:
# Compare graph properties for similar graphs 
#g <- graph.formula(A-B,B-C,C-D,D-E)
#g <- graph.formula(A-B,A-C,A-D,A-E,A-F)
g <- graph.formula(A-B,B-C,C-D,C-E,C-F,D-F,E-F)

# Uncomment to get a simple list of values for each node
# Degree
#degree(g,normalized=T)
# Closeness (inverse of average dist)
#closeness(g,normalized=T)
# Betweenness
#betweenness(g,normalized=T)
# Local cluster coefficient - Doesn't make sense for very simple examples
#transitivity(g, type="local")
# Eigenvector centrality
#evcent(g,scale=F)$vector

# Now rank them
order(degree(g))
order(closeness(g))
#order(transitivity(g,type="local"))
order(betweenness(g))
order(evcent(g,scale=T)$vector)

txt<-c("a","b","c","d")    # vector for labeling the graphs
par(mfrow=c(2,2),mar=c(.2,.2,.2,.2))
plot.igraph(g,vertex.size=100*degree(g,normalized=T),vertex.label=degree(g,normalized=T),layout=layout.auto,frame=T)
text(1,1,txt[1]) 
plot.igraph(g,vertex.size=100*closeness(g,normalized=T),vertex.label=closeness(g,normalized=T),layout=layout.auto,frame=T)
text(1,1,txt[2]) 
plot.igraph(g,vertex.size=100*betweenness(g,normalized=T),vertex.label=betweenness(g,normalized=T),layout=layout.auto,frame=T)
text(1,1,txt[3]) 
plot.igraph(g,vertex.size=100*evcent(g,scale=F)$vector,vertex.label=evcent(g,scale=F)$vector,layout=layout.auto,frame=T)
text(1,1,txt[4]) 

In [ ]:
?evcent

In [ ]:
# Centrality measures applied:
# Nodes with different roles in regulating 'information flow'

# A study found that people with low Eigenvector centrality but high Betweenness centrality
# in social networks are important 'gate keepers', while people with high Eigenvector centrality but low Betweenness 
# centrality have direct contact to important persons. So lets plot Eigenvector centrality against Betweenness 
# centrality to identify individuals with these qualities.

# Create a graph by taking the union of two different graphs
g1 <- barabasi.game(100, directed=F)
g2 <- barabasi.game(100, directed=F)
g <- g1 %u% g2
lay <- layout.fruchterman.reingold(g)
# Plot the eigevector and betweenness centrality
plot(evcent(g)$vector, betweenness(g))
text(evcent(g)$vector, betweenness(g), 0:100, cex=0.6, pos=4)

In [ ]:
V(g)[31]$color <- 'red'
V(g)[77]$color <- 'green'
plot(g, layout=lay, vertex.size=8, 
vertex.label.cex=0.6)

In [ ]:
# Modularity

# Use a community detection algorithm to determine the most densely 
# connected nodes in a graph.
 
g.community<-walktrap.community(test.graph)
 
# This algorithm uses random walks to find the most densely connected subgraphs.
 
members<-membership(g.community)
# The members() function picks out the membership vector 
# (list of nodes in the most densely connected subgraph) from 
# the communtiy object (e.g., walktrap community).
list(members)

par(mar=c(.1,.1,.1,.1))    # sets the edges of the plotting area
names(g.community)
plot(g.community, test.graph, layout=layout.fruchterman.reingold,
vertex.size=10,
vertex.label.cex=.5,
edge.arrow.size=.5)

In [ ]:
# Repeat with a simpler graph
test.graph.1 <- barabasi.game(n=50,power=1)
g.community<-walktrap.community(test.graph.1) 
members<-membership(g.community)

par(mar=c(.1,.1,.1,.1))    # sets the edges of the plotting area
names(g.community)
plot(g.community, test.graph.1, layout=layout.fruchterman.reingold,
vertex.size=10,
vertex.label.cex=.5,
edge.arrow.size=.5)